In [2]:
import pandas as pd
import numpy as np
import os
import sys
from dataclasses import dataclass, field

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
df = pd.read_csv('../data/toots_labeled.csv')
df.head()

,_id,user,content,y
0,111753937734745893,nowifeinthelaopobing,冷静下来仍然觉得增进自己贡献价值是正路，但我今年真的要学会为自己争取才可以，我不会再让了。,0
1,111753937703902969,marthadear,war neulich auf ner veranstaltung und (wieder ...,1
2,111753937695537419,IHasWisdom,Q:\tHow many marketing people does it take to ...,0
3,111753937678847359,Steffell,The everlasting romance of Education: from Yes...,0
4,111753937675453376,Deiru,I mean I called it right around when I barely ...,0


In [5]:
len(df)

392

In [7]:
df.describe()

,_id,y
count,3.920000e+02,392.000000
mean,1.117540e+17,0.038265
std,2.028599e+10,0.192081
min,1.117539e+17,0.000000
25%,1.117539e+17,0.000000
50%,1.117540e+17,0.000000
75%,1.117540e+17,0.000000
max,1.117540e+17,1.000000


In [14]:
val, occ = np.unique(df['y'], return_counts=True)
print(occ,'\n', occ/len(df)*100)
# Classes non équilibrées

[377  15] 
 [96.17346939  3.82653061]
